# Toronto Neighboorhood Project

<b>PART1<b>

In [1]:
#PULLING TABLE DATA FROM WIKI#
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
!pip install html-table-parser-python3
import pandas as pd
import numpy as np
import urllib.request
from pprint import pprint
from html_table_parser import HTMLTableParser


def url_get_contents(url):
    """ Opens a website and read its binary contents (HTTP Response Body) """
    req = urllib.request.Request(url=url)
    f = urllib.request.urlopen(req)
    return f.read()


def main():
    url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
    xhtml = url_get_contents(url).decode('utf-8')

    p = HTMLTableParser()
    p.feed(xhtml)
    global table
    table=p.tables[0]
    pprint(table)
    

if __name__ == '__main__':
    main()

/bin/bash: conda: command not found
[['Postal Code', 'Borough', 'Neighbourhood'],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'],
 ['M6A', 'North York', 'Lawrence Manor, Lawrence Heights'],
 ['M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"],
 ['M8A', 'Not assigned', 'Not assigned'],
 ['M9A', 'Etobicoke', 'Islington Avenue, Humber Valley Village'],
 ['M1B', 'Scarborough', 'Malvern, Rouge'],
 ['M2B', 'Not assigned', 'Not assigned'],
 ['M3B', 'North York', 'Don Mills'],
 ['M4B', 'East York', 'Parkview Hill, Woodbine Gardens'],
 ['M5B', 'Downtown Toronto', 'Garden District, Ryerson'],
 ['M6B', 'North York', 'Glencairn'],
 ['M7B', 'Not assigned', 'Not assigned'],
 ['M8B', 'Not assigned', 'Not assigned'],
 ['M9B',
  'Etobicoke',
  'West Deane Park, Princess Gardens, Martin Grove, Islington, Clo

In [2]:
tbl=table
features=tbl[0]
print(features)

['Postal Code', 'Borough', 'Neighbourhood']


In [3]:
# delete feature names from table
del tbl[0]
tbl

[['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'],
 ['M6A', 'North York', 'Lawrence Manor, Lawrence Heights'],
 ['M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"],
 ['M8A', 'Not assigned', 'Not assigned'],
 ['M9A', 'Etobicoke', 'Islington Avenue, Humber Valley Village'],
 ['M1B', 'Scarborough', 'Malvern, Rouge'],
 ['M2B', 'Not assigned', 'Not assigned'],
 ['M3B', 'North York', 'Don Mills'],
 ['M4B', 'East York', 'Parkview Hill, Woodbine Gardens'],
 ['M5B', 'Downtown Toronto', 'Garden District, Ryerson'],
 ['M6B', 'North York', 'Glencairn'],
 ['M7B', 'Not assigned', 'Not assigned'],
 ['M8B', 'Not assigned', 'Not assigned'],
 ['M9B',
  'Etobicoke',
  'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale'],
 ['M1C', 'Scarborough', 'Rouge Hill, Port Union, Highland Creek'],
 ['M

In [4]:
import pandas as pd
import numpy as np
pdTable = pd.DataFrame(data=tbl)
pdTable.columns= features
pdTable.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
a=pdTable[pdTable.Borough != 'Not assigned']
a=a.reset_index(drop=True)
a.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
a.shape

(103, 3)

---- end of Part1 ----

<b>PART2<b>

In [7]:
!pip install pgeocode
import pgeocode

nomi = pgeocode.Nominatim('ca')
print(nomi.query_postal_code("M5G").latitude)
print(nomi.query_postal_code("M5G").longitude)

43.6564
-79.38600000000001


In [8]:
##dummy latitude values , copied fro postal codes
lan=pd.DataFrame(columns=["dummy"])
lan["dummy"]=a["Postal Code"]
lon=pd.DataFrame(columns=["dumy"]) 
lon["dumy"]=a["Postal Code"]

for k in range(len(a['Postal Code'])):
    #print(k)

    lan["dummy"][k]=(nomi.query_postal_code(lan["dummy"][k]).latitude)
    lon["dumy"][k]=(nomi.query_postal_code(lon["dumy"][k]).longitude)

lan["dummy"]
lon["dumy"]


0       -79.33
1     -79.3148
2     -79.3626
3     -79.4504
4     -79.3889
        ...   
98    -79.5076
99     -79.383
100   -79.2505
101   -79.4939
102   -79.5231
Name: dumy, Length: 103, dtype: object

In [9]:
lan.rename(columns={"dummy": "Latitude"})
lon.rename(columns={"dumy": "Longitude"})

,Longitude
0,-79.33
1,-79.3148
2,-79.3626
3,-79.4504
4,-79.3889
...,...
98,-79.5076
99,-79.383
100,-79.2505
101,-79.4939


In [10]:
a['Latitude'] = lan
a['Longitude'] = lon
a.head(12)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.33
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.6662,-79.5282
6,M1B,Scarborough,"Malvern, Rouge",43.8113,-79.193
7,M3B,North York,Don Mills,43.745,-79.359
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7063,-79.3094
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783


---- end of Part2 ----

<b>PART3<b>

Filtering boroughs including 'toronto'

In [30]:
table_toronto = a[a['Borough'].str.contains('Toronto',regex=False)]

table_toronto=table_toronto.replace({43.7804: 43.662744, -79.2505: -79.321558})   ## I manually corrected M7Y coordinate here.
table_toronto
	

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655500,-79.362600
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.664100,-79.388900
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657200,-79.378300
15,M5C,Downtown Toronto,St. James Town,43.651300,-79.375600
19,M4E,East Toronto,The Beaches,43.678400,-79.294100
20,M5E,Downtown Toronto,Berczy Park,43.645600,-79.375400
24,M5G,Downtown Toronto,Central Bay Street,43.656400,-79.386000
25,M6G,Downtown Toronto,Christie,43.668300,-79.420500
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.649600,-79.383300
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.665500,-79.437800


Toronto map with neighbourhoods (in **toronto boroughs)

In [31]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(table_toronto['Latitude'],table_toronto['Longitude'],table_toronto['Borough'],table_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

**Using KMeans clustering for the neighbourhoods**

In [32]:
k=5
toronto_cluster = table_toronto.drop(['Postal Code','Borough','Neighbourhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_cluster)
kmeans.labels_
table_toronto.insert(0, 'Cluster Labels', kmeans.labels_)

In [33]:
table_toronto

,Cluster Labels,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655500,-79.362600
4,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.664100,-79.388900
9,4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657200,-79.378300
15,4,M5C,Downtown Toronto,St. James Town,43.651300,-79.375600
19,2,M4E,East Toronto,The Beaches,43.678400,-79.294100
20,4,M5E,Downtown Toronto,Berczy Park,43.645600,-79.375400
24,4,M5G,Downtown Toronto,Central Bay Street,43.656400,-79.386000
25,0,M6G,Downtown Toronto,Christie,43.668300,-79.420500
30,4,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.649600,-79.383300
31,3,M6H,West Toronto,"Dufferin, Dovercourt Village",43.665500,-79.437800


In [34]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(table_toronto['Latitude'], table_toronto['Longitude'], table_toronto['Neighbourhood'], table_toronto['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

End of Part 3